# Práctica 3. Optimización de hiperparámetros de MLP

In [5]:
from keras.datasets import mnist
import numpy as np
from sklearn.model_selection import train_test_split

(train_X, train_y), (test_X, test_y) = mnist.load_data()

vect_size = train_X.shape[1]*train_X.shape[2]

train_X = train_X.astype("float64")/255.0
train_X = train_X.reshape((train_X.shape[0], vect_size))

train_X, _, train_y, _ = train_test_split(train_X, train_y, train_size=10000, random_state=123, shuffle=True, stratify=train_y)

test_X = test_X.astype("float64")/255.0
test_X = test_X.reshape((test_X.shape[0], vect_size))

train_y.shape

(10000,)

In [6]:
from sklearn.neural_network import MLPClassifier

MLPClassifier().get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

param_grid = [
    {"hidden_layer_sizes": [(300,), (800,)], 
     "batch_size": [128, 512]}
]

mlp = MLPClassifier(
    random_state=123,
    batch_size=512,
    activation="relu",
    solver="adam",
    max_iter=50,
    early_stopping=True,
    verbose=True
)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=123)

search = GridSearchCV(estimator=mlp, param_grid=param_grid, scoring="accuracy", cv=cv, n_jobs=15)

search.fit(X=train_X, y=train_y)

Iteration 1, loss = 0.56052779
Validation score: 0.912000
Iteration 2, loss = 0.23711942
Validation score: 0.934000
Iteration 3, loss = 0.17363968
Validation score: 0.937000
Iteration 4, loss = 0.12354269
Validation score: 0.941000
Iteration 5, loss = 0.09543145
Validation score: 0.949000
Iteration 6, loss = 0.07217364
Validation score: 0.947000
Iteration 7, loss = 0.05398332
Validation score: 0.955000
Iteration 8, loss = 0.03792084
Validation score: 0.957000
Iteration 9, loss = 0.03004464
Validation score: 0.955000
Iteration 10, loss = 0.02366902
Validation score: 0.957000
Iteration 11, loss = 0.01682980
Validation score: 0.959000
Iteration 12, loss = 0.01420612
Validation score: 0.953000
Iteration 13, loss = 0.01305507
Validation score: 0.952000
Iteration 14, loss = 0.00869223
Validation score: 0.959000
Iteration 15, loss = 0.00730796
Validation score: 0.958000
Iteration 16, loss = 0.00595733
Validation score: 0.958000
Iteration 17, loss = 0.00526790
Validation score: 0.958000
Iterat

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=123),
             estimator=MLPClassifier(batch_size=512, early_stopping=True,
                                     max_iter=50, random_state=123,
                                     verbose=True),
             n_jobs=15,
             param_grid=[{'batch_size': [128, 512],
                          'hidden_layer_sizes': [(300,), (800,)]}],
             scoring='accuracy')

In [8]:
import pandas as pd

results_df = pd.DataFrame(search.cv_results_)
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("setting")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

,params,rank_test_score,mean_test_score,std_test_score
setting,,,,
"128_(800,)","{'batch_size': 128, 'hidden_layer_sizes': (800,)}",1,0.954633,0.004731
"512_(800,)","{'batch_size': 512, 'hidden_layer_sizes': (800,)}",2,0.950967,0.005951
"128_(300,)","{'batch_size': 128, 'hidden_layer_sizes': (300,)}",3,0.950600,0.005429
"512_(300,)","{'batch_size': 512, 'hidden_layer_sizes': (300,)}",4,0.945467,0.006059


In [9]:
from itertools import combinations
from math import factorial

import numpy as np
from scipy.stats import t


def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : float
        Variance-corrected standard deviation of the set of differences.
    """
    # kr = k times r, r times repeated k-fold crossvalidation,
    # kr equals the number of times the model was evaluated
    kr = len(differences)
    corrected_var = np.var(differences, ddof=1) * (1 / kr + n_test / n_train)
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return t_stat, p_val

model_scores = results_df.filter(regex=r"split\d*_test_score")

n = model_scores.shape[0]
df = n - 1
n_train = len(list(cv.split(train_X, train_y))[0][0])
n_test = len(list(cv.split(train_X, train_y))[0][1])



n_comparisons = factorial(len(model_scores)) / (
    factorial(2) * factorial(len(model_scores) - 2)
)
pairwise_t_test = []

for model_i, model_k in combinations(range(len(model_scores)), 2):
    model_i_scores = model_scores.iloc[model_i].values
    model_k_scores = model_scores.iloc[model_k].values
    differences = model_i_scores - model_k_scores
    t_stat, p_val = compute_corrected_ttest(differences, df, n_train, n_test)
    p_val *= n_comparisons  # implement Bonferroni correction
    # Bonferroni can output p-values higher than 1
    p_val = 1 if p_val > 1 else p_val
    pairwise_t_test.append(
        [model_scores.index[model_i], model_scores.index[model_k], t_stat, p_val]
    )

pairwise_comp_df = pd.DataFrame(
    pairwise_t_test, columns=["model_1", "model_2", "t_stat", "p_val"]
).round(3)

pairwise_comp_df

,model_1,model_2,t_stat,p_val
0,"128_(800,)","512_(800,)",1.742,0.540
1,"128_(800,)","128_(300,)",1.603,0.622
2,"128_(800,)","512_(300,)",3.673,0.105
3,"512_(800,)","128_(300,)",0.163,1.000
4,"512_(800,)","512_(300,)",2.843,0.197
5,"128_(300,)","512_(300,)",2.446,0.276


In [10]:
model_scores

,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score
setting,,,,,,,,,,,,,,,
"128_(800,)",0.944,0.9575,0.9530,0.9585,0.9575,0.959,0.9525,0.9565,0.9490,0.9540,0.9465,0.9590,0.961,0.9565,0.9550
"512_(800,)",0.934,0.9585,0.9505,0.9550,0.9570,0.952,0.9520,0.9530,0.9420,0.9495,0.9505,0.9560,0.953,0.9535,0.9480
"128_(300,)",0.940,0.9580,0.9460,0.9585,0.9500,0.947,0.9505,0.9515,0.9425,0.9480,0.9535,0.9520,0.960,0.9525,0.9490
"512_(300,)",0.929,0.9530,0.9460,0.9510,0.9485,0.947,0.9500,0.9425,0.9380,0.9450,0.9460,0.9465,0.954,0.9410,0.9445


## Pairwise comparison of all models: Bayesian approach

In [11]:

# initialize random variable
t_post = t(
    df, loc=np.mean(differences), scale=corrected_std(differences, n_train, n_test)
)

rope_interval = [-0.01, 0.01]

pairwise_bayesian = []

for model_i, model_k in combinations(range(len(model_scores)), 2):
    model_i_scores = model_scores.iloc[model_i].values
    model_k_scores = model_scores.iloc[model_k].values
    differences = model_i_scores - model_k_scores
    t_post = t(
        df, loc=np.mean(differences), scale=corrected_std(differences, n_train, n_test)
    )
    worse_prob = t_post.cdf(rope_interval[0])
    better_prob = 1 - t_post.cdf(rope_interval[1])
    rope_prob = t_post.cdf(rope_interval[1]) - t_post.cdf(rope_interval[0])

    pairwise_bayesian.append([worse_prob, better_prob, rope_prob])

pairwise_bayesian_df = pd.DataFrame(
    pairwise_bayesian, columns=["worse_prob", "better_prob", "rope_prob"]
).round(3)

pairwise_comp_df = pairwise_comp_df.join(pairwise_bayesian_df)
pairwise_comp_df

,model_1,model_2,t_stat,p_val,worse_prob,better_prob,rope_prob
0,"128_(800,)","512_(800,)",1.742,0.540,0.004,0.029,0.968
1,"128_(800,)","128_(300,)",1.603,0.622,0.006,0.049,0.945
2,"128_(800,)","512_(300,)",3.673,0.105,0.002,0.380,0.617
3,"512_(800,)","128_(300,)",0.163,1.000,0.010,0.012,0.979
4,"512_(800,)","512_(300,)",2.843,0.197,0.002,0.051,0.947
5,"128_(300,)","512_(300,)",2.446,0.276,0.003,0.052,0.946
